# Développez un chatbot pour réserver des vacances - LUIS

In [1]:
import pandas as pd
import json
import time
import os
import requests

from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.language.luis.authoring.models import ApplicationCreateObject
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient

In [4]:
AUTHORING_KEY_LUIS = '7117b3e146fa421094176f4424a80ece'
AUTHORING_ENDPOINT_LUIS = 'https://p10luis-authoring.cognitiveservices.azure.com/'

# Instantiate a LUIS client
client = LUISAuthoringClient(AUTHORING_ENDPOINT_LUIS, CognitiveServicesCredentials(AUTHORING_KEY_LUIS))

### Building & Training LUIS

In [9]:
app_name = "Booking Bot"
app_desc = "Flight booking app built with LUIS Python SDK."
app_version = "0.0"
app_locale = "en-us"

appDefinition = ApplicationCreateObject (name=app_name, initial_version_id=app_version, culture=app_locale)
app_id = client.apps.add(appDefinition)

print("Created LUIS app with ID {}".format(app_id))

Created LUIS app with ID e6eb0e67-2cf9-45aa-9ec5-d3f808012404


In [11]:
dst_city_entity_id = client.model.add_entity(
        app_id, app_version, name="dst_city")
print("destinationEntityId {} added.".format(dst_city_entity_id))

or_city_entity_id = client.model.add_entity(
        app_id, app_version, name="or_city")
print("classEntityId {} added.".format(or_city_entity_id))

str_date_entity_id = client.model.add_entity(
        app_id, app_version, name="str_date")
print("flightEntityId {} added.".format(str_date_entity_id))

end_date_entity_id = client.model.add_entity(
        app_id, app_version, name="end_date")
print("flightEntityId {} added.".format(end_date_entity_id))

budget_entity_id = client.model.add_entity(
        app_id, app_version, name="budget")
print("flightEntityId {} added.".format(budget_entity_id))

print()

intent_id = client.model.add_intent(app_id, app_version, "BookFlight")
print("Intent BookFlight {} added.".format(intent_id))

destinationEntityId 1ee76ed5-51ee-458e-9d4c-b3d424d5ee02 added.
classEntityId d084d3dc-e306-4129-9644-b5bab245480e added.
flightEntityId ad8fd3a0-737c-4e1f-a9ed-9d0eaf750b01 added.
flightEntityId b9b47fc3-816f-41ac-ac76-0c0d8da03aed added.
flightEntityId 7e000f42-5c2a-4c58-98e7-b4e7bcaf62d4 added.
Intent BookFlight 38878dd0-7849-46a0-bfea-a1b4832135f4 added.


The data should look like :

         {
             \'text\':\'user text.\',
             \'intent_name\'=\'Example\',
             \'entity_labels\':[
                 {
                     \'entity_name':\'wordText\',
                     \'start_char_index\':3,
                     \'end_char_index\':8
                 }
             ]
         }

In [ ]:
#Read & Convert DATA

JSON_FILE_PATH = "D:/DATA/FORMATION_IA/P10_chaleyssin_nicolas/dataset/frames.json"
df = pd.read_json(JSON_FILE_PATH)
all_turn_list = df["turns"].tolist()

def format_data_luis(intent, text, labels):

    text = text.lower()
    def entity_format(name, value):

        # we retrieve the size and the beginning of the index of the value on the text to have the beginning and the end of the value in the text
        value = value.lower()
        start = text.index(value)
        return dict(entity_name=name, start_char_index=start,
                    end_char_index=start + len(value))

    return dict(text=text, intent_name=intent,
                    entity_labels=[entity_format(n, v) for (n, v) in labels])

def get_key_value(arg):
    
    list_key = ['dst_city','or_city','str_date','end_date','budget']
    if "key" in arg and "val" in arg:
        if arg["key"] in list_key and arg['val'] != "-1" and arg['val'] != None:
            return (arg["key"],arg["val"])
        
    return None

luis_data = []
for conversation in all_turn_list:
    for message in conversation:
        # Only retrieve user messages
        if message["author"] == "user":
            text = message["text"]
            acts = message["labels"]["acts"]
            conversation_list = []
            for act in acts:
                if act["args"]:
                    for arg in act["args"]:
                        if not arg["key"] == "ref":
                            data = get_key_value(arg)
                            if data:
                                conversation_list.append(data)
                        else:
                            annotations_list = arg['val'][0]['annotations']
                            if annotations_list:
                                for annotation_value in annotations_list:
                                    data = get_key_value(annotation_value)
                                if data:
                                    conversation_list.append(data)
            if conversation_list:
                luis.append(format_data_luis("BookFlight", text, conversation_list))

In [14]:
#exemple
luis_data[0]

{'text': "i'd like to book a trip to atlantis from caprica on saturday, august 13, 2016 for 8 adults. i have a tight budget of 1700.",
 'intent_name': 'BookFlight',
 'entity_labels': [{'entity_name': 'dst_city',
   'start_char_index': 27,
   'end_char_index': 35},
  {'entity_name': 'or_city', 'start_char_index': 41, 'end_char_index': 48},
  {'entity_name': 'str_date', 'start_char_index': 52, 'end_char_index': 77},
  {'entity_name': 'budget', 'start_char_index': 117, 'end_char_index': 121}]}

In [16]:
#Training Luis
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(luis_data, test_size=0.05)
print("train_set shape : {} test_set shape : {}\n".format(train_set.shape, test_set.shape))

for i in range(0, len(train_set), 100):
    j = (i + 100)
    if j > len(final_data):
        j = len(final_data)
    # We add 100 by 100 because requests are blocked beyond 100
    client.examples.batch(app_id,app_version,train_set[i:j])
    
client.train.train_version(app_id, app_version)
waiting = True

while waiting:
    info = client.train.get_status(app_id, app_version)

    # get_status returns a list of training statuses, one for each model. 
    # Loop through them and make sure all are done.
    waiting = any(map(lambda x: 'Queued' == x.details.status or 'InProgress' == x.details.status, info))
    if waiting:
        print ("Waiting 10 seconds for training to complete...")
        time.sleep(10)
    else: 
        print ("trained")
        waiting = False

Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
trained


In [18]:
# Update the app
client.apps.update_settings(app_id, is_public=True)

# Get the endpoint
responseEndpointInfo = client.apps.publish(app_id, app_version, is_staging=False)
endpoint = responseEndpointInfo.endpoint_url + "?subscription-key=" + AUTHORING_KEY_LUIS + "&q="

print("Your app is published. You can now go to test it on\n{}".format(endpoint))

Your app is published. You can now go to test it on
https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=


### Luis model

In [6]:
import pandas as pd
import json
import time
import os
import requests

from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.language.luis.authoring.models import ApplicationCreateObject
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient

In [3]:
class LuisApp:
    def __init__(self,authoring_endpoint_luis,authoring_key_luis,
                list_entities=['dst_city','or_city','str_date','end_date','budget'],
                list_intents=['BookFlight'],
                runtime_endpoint_luis = None, runtime_key_luis = None):
        '''Makes it easier to handle LUIS applications
         Parameters:
             authoring_endpoint_luis
             authoring_key_luis
             list_entities
             list_intents
             runtime_endpoint_luis
             runtime_key_luis
         Returns:
             None
         '''
        self.AUTHORING_ENDPOINT = authoring_endpoint_luis
        self.AUTHORING_KEY = authoring_key_luis
        self.list_entities = list_entities
        self.list_intents = list_intents

        self.app_created = False
        self.app_published = False

        self.client = LUISAuthoringClient(self.AUTHORING_ENDPOINT, CognitiveServicesCredentials(self.AUTHORING_KEY))

        # information
        self.app_id = None
        self.app_version = None

        self.app_locale = None
        self.app_desc = None
        self.app_name = None
        
        self.endpoint=None
        self.publish_version = None

        self.RUNTIME_ENDPOINT = runtime_endpoint_luis
        self.RUNTIME_KEY = runtime_key_luis

        if self.RUNTIME_ENDPOINT and self.RUNTIME_KEY:
            self.client_runtime = LUISRuntimeClient(self.RUNTIME_ENDPOINT, CognitiveServicesCredentials(self.RUNTIME_KEY))
        else:
            self.client_runtime = None

    def get_app(self,app_id):
        '''
        '''
        information = self.client.apps.get(app_id)

        self.app_created = True
        if information.endpoints:
            self.endpoint=information.endpoints["PRODUCTION"]["endpointUrl"]
            self.endpoint = self.endpoint +"?subscription-key=" + self.AUTHORING_KEY + "&q="
            self.app_published = True
        self.app_id = app_id
        self.app_version = information.active_version
        self.app_locale = information.culture
        self.app_desc = information.description
        self.app_name = information.name

    def evaluate(self,test_data):
        ''' 
         Returns:
             all_score (list -> float): corresponds to the model accuracy scores for each text
             mean_score (float): corresponds to the average score obtained
        '''
        # Compute with predict if possible
        if self.app_created and self.app_published:
            all_score = []
            for data in test_data:
                y_pred = self.predict(data["text"])
                print(y_pred)
                y_true = self._format_y_true(data)
                score = self._calcul_score_accuracy(y_pred,y_true)
                all_score.append(score)
            mean_score = sum(all_score) / len(all_score)
            return all_score, mean_score
        else:
            raise Exception("The application is not available, check its creation or deployment.")

    def _format_y_true(self,y_true_data):
        y_true = {}
        text = y_true_data["text"]
        for entities in y_true_data["entity_labels"]:
            y_true[entities["entity_name"]] = text[entities["start_char_index"]:entities["end_char_index"]]
        
        return y_true

    def _calcul_score_accuracy(self,y_pred,y_true):
        score = 0
        for k,v in y_true.items():
            if k in y_pred:
                score += self._evaluate_value(v,y_pred[k])
            else:
                score += 0
        # accuracy
        accuracy_score = 1/len(y_true.keys()) * score
        return accuracy_score

    def _evaluate_value(self,data_true,data_pred):
        # Can be improved by using for example levenshtein's method
         # or another similarity method
        if data_true == data_pred:
            return 1
        elif data_true in data_pred:
            return 0.5
        else:
            return 0                    
        
    def predict(self,query,get_intent = False, timeout_seconde = 10):
        '''
        '''
        if self.client_runtime:
            # Using runtime client SDK
            response = self.client_runtime.prediction.get_slot_prediction(app_id=self.app_id,
                                                                        slot_name="production",
                                                                        prediction_request=query)
            y_pred = response.prediction.entities
            y_pred = {k:v[0] for k,v in y_pred.items()}
            if get_intent:
                intent = response.prediction.intents.keys()
                return y_pred, intent
            else:
                return y_pred
            
        else:
            response = None
            start = time.time()
            duration = 0
            # We loop until we have a response but a timeout if no valid response
            while response == None and duration <= timeout_seconde:
                time.sleep(0.5)
                response = requests.get(self.endpoint + query)
                print(self.endpoint + query, response.json())
                duration = time.time() - start
            response = response.json()
            y_pred = {}
            if "entities" in response:
                for entity in response["entities"]:
                    data = entity
                    key = data["type"]
                    value = data["entity"]
                    y_pred[key] = value
                    
                if get_intent:
                    intent = response["topScoringIntent"]["intent"]
                    return y_pred, intent
                else:
                    return y_pred

In [4]:
AUTHORING_KEY_LUIS = '7117b3e146fa421094176f4424a80ece'
AUTHORING_ENDPOINT_LUIS = 'https://p10luis-authoring.cognitiveservices.azure.com/'
app_id = 'e6eb0e67-2cf9-45aa-9ec5-d3f808012404'
#RUNTIME_ENDPOINT = "https://bookingbot.cognitiveservices.azure.com/"
#RUNTIME_KEY = "ef06751ee36e41da9821c0bc425cf77b"
luis_app = LuisApp(AUTHORING_ENDPOINT_LUIS, AUTHORING_KEY_LUIS)
luis_app.get_app(app_id)

In [21]:
# Compute the metrics on the test set
all_score, score = luis_app.evaluate(test_set)

https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=i want to go to monterrey from tofino {'query': 'i want to go to monterrey from tofino', 'topScoringIntent': {'intent': 'BookFlight', 'score': 0.99998474}, 'entities': [{'entity': 'monterrey', 'type': 'dst_city', 'startIndex': 16, 'endIndex': 24, 'score': 0.97040266}, {'entity': 'tofino', 'type': 'or_city', 'startIndex': 31, 'endIndex': 36, 'score': 0.99529004}]}
{'dst_city': 'monterrey', 'or_city': 'tofino'}
https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=hi so i need to go to leon i will leave atlanta and i will travel between september 13th and 25th got it??? {'query': 'hi so i need to go to leon i will leave atlanta and i will travel between september 13th and 25th got it???', 'topScoringIntent': {'intent': 'BookFlight', 'score': 0.

https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=i don't want to spend any more than $54700 {'query': "i don't want to spend any more than $54700", 'topScoringIntent': {'intent': 'BookFlight', 'score': 0.99987936}, 'entities': [{'entity': '$ 54700', 'type': 'budget', 'startIndex': 36, 'endIndex': 41, 'score': 0.9946706}]}
{'budget': '$ 54700'}
https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=by september 20 {'query': 'by september 20', 'topScoringIntent': {'intent': 'BookFlight', 'score': 0.9999962}, 'entities': []}
{}
https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=hey buddy. i have some work to do
looking to domesticate a savage little tea cup yorkie in pittsburgh
i am available bet

https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=can you find me a five star, even if i have to get there a week in advance? i can spend up to $6200 if i have to {'query': 'can you find me a five star, even if i have to get there a week in advance? i can spend up to $6200 if i have to', 'topScoringIntent': {'intent': 'BookFlight', 'score': 0.99988556}, 'entities': [{'entity': '$ 6200', 'type': 'budget', 'startIndex': 94, 'endIndex': 98, 'score': 0.9906067}]}
{'budget': '$ 6200'}
https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=3000! {'query': '3000!', 'topScoringIntent': {'intent': 'BookFlight', 'score': 0.9999981}, 'entities': [{'entity': '3000 !', 'type': 'budget', 'startIndex': 0, 'endIndex': 4, 'score': 0.99891883}]}
{'budget': '3000 !'}
https://westeurope.api.cognitive.microsoft.c

https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=i also had sao paulo in my mind.. {'query': 'i also had sao paulo in my mind..', 'topScoringIntent': {'intent': 'BookFlight', 'score': 0.99997854}, 'entities': [{'entity': 'sao paulo', 'type': 'dst_city', 'startIndex': 11, 'endIndex': 19, 'score': 0.9913951}]}
{'dst_city': 'sao paulo'}
https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=hello, i am looking to book a vacation from gotham city to mos eisley for $2100. {'query': 'hello, i am looking to book a vacation from gotham city to mos eisley for $2100.', 'topScoringIntent': {'intent': 'BookFlight', 'score': 0.9999428}, 'entities': [{'entity': '$ 2100 .', 'type': 'budget', 'startIndex': 74, 'endIndex': 79, 'score': 0.97884446}, {'entity': 'mos eisley', 'type': 'dst_city', 'startIndex': 5

https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=i’d really like to get there earlier if possible. {'query': 'i’d really like to get there earlier if possible.', 'topScoringIntent': {'intent': 'BookFlight', 'score': 0.99997663}, 'entities': [{'entity': 'earlier', 'type': 'str_date', 'startIndex': 29, 'endIndex': 35, 'score': 0.83090204}]}
{'str_date': 'earlier'}
https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=sl to rome, what hotels you got? {'query': 'sl to rome, what hotels you got?', 'topScoringIntent': {'intent': 'BookFlight', 'score': 0.9999833}, 'entities': [{'entity': 'rome ,', 'type': 'dst_city', 'startIndex': 6, 'endIndex': 10, 'score': 0.99857575}]}
{'dst_city': 'rome ,'}
https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subsc

...

https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=we can go to phoenix and borrow my sister’s mini van - we’ll make a little road trip out of it! {'query': 'we can go to phoenix and borrow my sister’s mini van - we’ll make a little road trip out of it!', 'topScoringIntent': {'intent': 'BookFlight', 'score': 0.9999027}, 'entities': [{'entity': 'phoenix', 'type': 'dst_city', 'startIndex': 13, 'endIndex': 19, 'score': 0.96464616}]}
{'dst_city': 'phoenix'}
https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=ok i would like to book this one. can i get business seats and still stay within my budget? {'query': 'ok i would like to book this one. can i get business seats and still stay within my budget?', 'topScoringIntent': {'intent': 'BookFlight', 'score': 0.9999418}, 'entities': []}
{}
https://w

https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=oh well, next time, book me to vitoria then and i'll see what's happening there {'query': "oh well, next time, book me to vitoria then and i'll see what's happening there", 'topScoringIntent': {'intent': 'BookFlight', 'score': 0.9999356}, 'entities': [{'entity': 'vitoria', 'type': 'dst_city', 'startIndex': 31, 'endIndex': 37, 'score': 0.9981497}]}
{'dst_city': 'vitoria'}
https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=would by chance have one leaving on the 19th {'query': 'would by chance have one leaving on the 19th', 'topScoringIntent': {'intent': 'BookFlight', 'score': 0.99997807}, 'entities': [{'entity': '19th', 'type': 'str_date', 'startIndex': 40, 'endIndex': 43, 'score': 0.9952639}]}
{'str_date': '19th'}
https://westeurope.api.co

https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=let's try la paz! {'query': "let's try la paz!", 'topScoringIntent': {'intent': 'BookFlight', 'score': 0.9999862}, 'entities': [{'entity': 'la paz !', 'type': 'dst_city', 'startIndex': 10, 'endIndex': 16, 'score': 0.915717}]}
{'dst_city': 'la paz !'}
https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=oh. well okay, economy is fine. is it booked? {'query': 'oh. well okay, economy is fine. is it booked?', 'topScoringIntent': {'intent': 'BookFlight', 'score': 0.999969}, 'entities': []}
{}
https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=no to august 30th but what about august 27? {'query': 'no to august 30th but what about august 27?', 'topSc

https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=i’m free from the 13th until the 21st of september {'query': 'i’m free from the 13th until the 21st of september', 'topScoringIntent': {'intent': 'BookFlight', 'score': 0.99997234}, 'entities': [{'entity': '21st of september', 'type': 'end_date', 'startIndex': 33, 'endIndex': 49, 'score': 0.9975403}, {'entity': '13th', 'type': 'str_date', 'startIndex': 18, 'endIndex': 21, 'score': 0.8026641}]}
{'end_date': '21st of september', 'str_date': '13th'}
https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=i have 0 budget, 16 days vacay, leaving from barcelona and am looking for possible destinations {'query': 'i have 0 budget, 16 days vacay, leaving from barcelona and am looking for possible destinations', 'topScoringIntent': {'intent': 'BookFlight

https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=no but i guess i can increase my budget by 200 {'query': 'no but i guess i can increase my budget by 200', 'topScoringIntent': {'intent': 'BookFlight', 'score': 0.9999733}, 'entities': [{'entity': 'increase my budget by 200', 'type': 'budget', 'startIndex': 21, 'endIndex': 45, 'score': 0.9135722}]}
{'budget': 'increase my budget by 200'}


In [24]:
print("Accuracy score of the model is : {}%".format(round(score*100,2)))

Accuracy score of the model is : 60.94%


In [9]:
#try it
TEXT = "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700."

# Launch the prediction
luis_app.predict(TEXT)

https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/e6eb0e67-2cf9-45aa-9ec5-d3f808012404?subscription-key=7117b3e146fa421094176f4424a80ece&q=I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700. {'query': "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.", 'topScoringIntent': {'intent': 'BookFlight', 'score': 0.9998965}, 'entities': [{'entity': '1700 .', 'type': 'budget', 'startIndex': 117, 'endIndex': 121, 'score': 0.9934712}, {'entity': 'atlantis', 'type': 'dst_city', 'startIndex': 27, 'endIndex': 34, 'score': 0.9582799}, {'entity': 'caprica', 'type': 'or_city', 'startIndex': 41, 'endIndex': 47, 'score': 0.9854948}, {'entity': 'saturday , august 13 , 2016', 'type': 'str_date', 'startIndex': 52, 'endIndex': 76, 'score': 0.99888}]}


{'budget': '1700 .',
 'dst_city': 'atlantis',
 'or_city': 'caprica',
 'str_date': 'saturday , august 13 , 2016'}